In [90]:
%load_ext autoreload
# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import math
import EoN
import seaborn as sns
import time

# New Imports
#from ctrace.contact_tracing import *
#from ctrace.constraint import *
#from ctrace.solve import *
#from ctrace.simulation import *
#from ctrace.restricted import *
from ctrace.simulation import *
from ctrace.dataset import *
from ctrace.recommender import *
from ctrace.problem import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
G = load_graph("montgomery")

In [4]:
I = set()
with open(PROJECT_ROOT / "data" / "SIR_Cache" / "t7.json", 'r') as infile:
    j = json.load(infile)
    (S, infected_queue, R) = (j["S"], j["I_Queue"], j["R"])

    # Make infected_queue a list of sets
    infected_queue = [set(s) for s in infected_queue]
    I = I.union(*infected_queue)
    I = list(I)

In [93]:
#this is how the code is used
state = SimulationState(G, (S,I,R), (S,I,R), 2000, .078,.5,1,1,1)
agent = Random

while len(state.SIR_known.SIR[1]) != 0:
    #print(len(state.SIR_known.SIR[1]))
    to_quarantine = agent(state)
    state.step(to_quarantine)
    
    print(state.SIR_known.SIR.I in state.SIR_real.SIR.I)
#print(len(state.SIR_known.SIR[1]))
print()
print(len(state.SIR_known.SIR[2]))

False
False
False
False
False
False
False
False
False

41734


In [76]:
print([1].concat([2]))

AttributeError: 'list' object has no attribute 'concat'

In [27]:
state.SIR_known.quarantined

([], [], [])

In [28]:
len(state.SIR_real.SIR[0] + state.SIR_real.SIR[1] + state.SIR_real.SIR[2])

75457

In [43]:
agents = [NoIntervention, Random, Degree, DegGreedy]

In [44]:
for agent in agents:
    state = SimulationState(G, (S,I,R), (S,I,R), 2000, .078,1,1, 1,1)

    while len(state.SIR_known.SIR[1]) != 0:
        to_quarantine = agent(state)
        state.step(to_quarantine)

    print(agent.__name__ + " " + str(len(state.SIR_known.SIR[2])))


NoIntervention 36117
Random 26046
Degree 21258
DegGreedy 18275
